# Advanced Spectrum List Patterns and Workarounds

`muler` is built on specutils, which has many spectrum list operations, but not everything we'd ideally want.  In this notebook, we introduce a few workarounds to deal with common patterns.

In [1]:
%config Completer.use_jedi = False

In [2]:
from muler.igrins import IGRINSSpectrum, IGRINSSpectrumList
from specutils import Spectrum1D, SpectrumList
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format='retina'

Simply tell the IGRINS Spectrum where your file is located.

In [3]:
path = 'https://github.com/OttoStruve/muler_example_data/raw/main/IGRINS/01_IGRINS_test_data/'
filename1='SDCH_20201202_0059.spec_a0v.fits'
full_path = path + filename1

In [4]:
#spec_list = IGRINSSpectrumList.read(full_path).normalize()

In [11]:
spec = IGRINSSpectrum(file=full_path)

wavelength = spec.wavelength
flux = spec.flux

roundtrip_spec = IGRINSSpectrum(spectral_axis=wavelength, flux=flux)

The spectrum metadata appears to be missing.  The functionality of muler may be impaired without metadata.  See discussion at https://github.com/OttoStruve/muler/issues/79.


In [12]:
spec.normalize()

The spectrum metadata appears to be missing.  The functionality of muler may be impaired without metadata.  See discussion at https://github.com/OttoStruve/muler/issues/79.


<Spectrum1D(flux=<Quantity [       nan, 6.08261594, 9.37395966, ...,        nan,        nan,
                  nan]>, spectral_axis=<SpectralAxis 
   (observer to target:
      radial_velocity=0.0 km / s
      redshift=0.0)
  [16461.49654964, 16461.62499595, 16461.7534228 , ..., 16692.50318342,
   16692.60471502, 16692.70623909] Angstrom>, uncertainty=StdDevUncertainty([       nan, 2.34105076, 6.76584695, ...,        nan,
                          nan,        nan]))>

In [ ]:
spec_list.remove_nans().trim_edges().normalize(order_index=15).plot(color=None, ylo=0, yhi=1.5);

## Unifying H and K bands into a single `IGRINSSpectrumList`

In [ ]:
filename2 = 'SDCK_20201202_0059.spec_a0v.fits'
spec_list2 = IGRINSSpectrumList.read(path+filename2).normalize()

The `extend()` method works **in-place**, overriding the first argument!  We have to copy the `spec_list` if we want to keep it pristine:

In [ ]:
import copy

In [ ]:
full_H_and_K_spectrum = copy.deepcopy(spec_list)

In [ ]:
full_H_and_K_spectrum.extend(spec_list2)

In [ ]:
len(spec_list), len(spec_list2), len(full_H_and_K_spectrum)

In [ ]:
full_H_and_K_spectrum.remove_nans().trim_edges().flatten().plot(color=None, ylo=0.8, yhi=1.2);

## Workarounds

Here we introduce workarounds to three friction points.


### Issue 1: Slicing a `IGRINSSpectrumList`

First, we want to be able to index into the `IGRINSSpectrumList` in the same way you would with a regular old Python List, and still *get back* an `IGRINSSpectrumList`.

In [ ]:
type(spec_list)

In [ ]:
sublist = spec_list[4:7]

In [ ]:
type(sublist)

No! We want this sublist to be an `IGRINSSpectrumList`

In [ ]:
sublist_IGRINS = IGRINSSpectrumList(sublist)

In [ ]:
type(sublist_IGRINS)

Yay! This workaround achieved our goal.

### Issue 2: Create an `IGRINSSpectrum` from a bare (`flux`, `wavelength`) array

Often we have already preprocessed a spectrum and have it in-hand as a pair of wavelength coordinates and flux values.  We wish to turn this most elemental format of a spectrum into a `IGRINSSpectrum`.

In [ ]:
import astropy.units as u

In [ ]:
whole_spectrum = spec_list.remove_nans().trim_edges().normalize().stitch()

In [ ]:
len(whole_spectrum.flux.value)

In [ ]:
wavelength = whole_spectrum.wavelength.value
flux = whole_spectrum.flux.value

Can we "round-trip" this pair of coordinates and flux values back into an `IGRINSSpectrum`?

In [ ]:
roundtrip_spec = IGRINSSpectrum(spectral_axis=wavelength*u.Angstrom, 
                                flux=flux*u.dimensionless_unscaled, )

In [ ]:
type(roundtrip_spec)

Some methods will work out-of-the-box:

In [ ]:
roundtrip_spec.normalize() # works fine!

But others do not!

In [ ]:
output = whole_spectrum.barycentric_correct() # works fine!

Watch out! This next line will not work!

In [ ]:
force = False

if force:
    roundtrip_spec.barycentric_correct() # Oops! Will not work!